# Big Data
# BD03.5-MostPopularSuperHero



Tenemos el dataset Marvel+Graph, este archivo contiene en cada registro lo siguiente: el primer codigo el super heroe principal y luego los codigos de heroes que aparecen junto a el, es decir en el mismo libro. EL objetivo es encontrar el super heroe mas famoso que corresponde al que tenga la mayor catidad de co-appariciones. 

In [1]:
# Esto solo lo utilizamos para instalar las librerias 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz   
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

## <font color='green'>**Ejercicio 1**</font>

Las instrucciones
1. Separe el ID del heroe del inicio de la linea.
2. Cuente cuántos números separados por espacios hay en la línea. Reste uno al numero total de conecciones.
3. Agrupe por ID de héroe para sumar conexiones divididas en varias líneas
4. Ordene por el total de conecciones
5. Filtre el conjunto de datos de nombres por el ID de héroe más popular para buscar el nombre.

Los archivos a utilizar 
1. Marvel+Names
2. Marvel+Graph

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [42]:
ids = spark.sparkContext.textFile("/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/Marvel+Graph")

In [64]:
ids.take(1)

['5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 ']

In [117]:
import pyspark.sql.functions as F
ids_ = ids.map(lambda x: x.split(" ")).map(lambda x: (x, x[0], len(x)-1))

In [118]:
ids_.take(1)

[(['5988',
   '748',
   '1722',
   '3752',
   '4655',
   '5743',
   '1872',
   '3413',
   '5527',
   '6368',
   '6085',
   '4319',
   '4728',
   '1636',
   '2397',
   '3364',
   '4001',
   '1614',
   '1819',
   '1585',
   '732',
   '2660',
   '3952',
   '2507',
   '3891',
   '2070',
   '2239',
   '2602',
   '612',
   '1352',
   '5447',
   '4548',
   '1596',
   '5488',
   '1605',
   '5517',
   '11',
   '479',
   '2554',
   '2043',
   '17',
   '865',
   '4292',
   '6312',
   '473',
   '534',
   '1479',
   '6375',
   '4456',
   ''],
  '5988',
  49)]

In [122]:
ids_df = ids_.toDF()
ids_df.show()

+--------------------+----+---+
|                  _1|  _2| _3|
+--------------------+----+---+
|[5988, 748, 1722,...|5988| 49|
|[5989, 4080, 4264...|5989| 41|
|[5982, 217, 595, ...|5982| 43|
|[5983, 1165, 3836...|5983| 15|
|[5980, 2731, 3712...|5980| 25|
|[5981, 3569, 5353...|5981| 18|
|[5986, 2658, 3712...|5986|143|
|[5987, 2614, 5716...|5987| 82|
|[5984, 590, 4898,...|5984| 42|
|[5985, 3233, 2254...|5985| 20|
|[6294, 4898, 1127...|6294| 14|
|[270, 2658, 3003,...| 270| 43|
|[271, 4935, 5716,...| 271| 10|
|[272, 2717, 4363,...| 272| 46|
|[273, 1165, 5013,...| 273| 59|
|[274, 3920, 5310,...| 274|411|
|[275, 4366, 3373,...| 275| 48|
|[276, 2277, 5251,...| 276| 16|
|[277, 1068, 3495,...| 277| 17|
|[278, 1145, 667, ...| 278|124|
+--------------------+----+---+
only showing top 20 rows



In [131]:
grouped_id = ids_df.groupBy("_2").agg(func.sum("_3").alias("suma"))
grouped_id.show()

+----+----+
|  _2|suma|
+----+----+
| 691|   7|
|1159|  12|
|3959| 143|
|1572|  36|
|2294|  15|
|1090|   5|
|3606| 172|
|3414|   8|
| 296|  18|
|4821|  17|
|2162|  42|
|1436|  10|
|1512|  12|
|3517|  14|
|5613| 127|
|5657|  55|
|1669|   7|
| 451|  19|
|2464|   6|
|2275|  13|
+----+----+
only showing top 20 rows



In [83]:
from operator import add
ids_.reduceByKey(add).take(5)

[('5989', 41), ('5982', 43), ('5980', 25), ('5984', 42), ('270', 43)]

In [132]:
ordered_id = grouped_id.sort(grouped_id.suma.desc())

In [135]:
most_popular_id = ordered_id.first()[0]

In [76]:
names = spark.sparkContext.textFile("/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/Marvel+Names")

In [77]:
names.take(2)

['1 "24-HOUR MAN/EMMANUEL"', '2 "3-D MAN/CHARLES CHAN"']

In [78]:
names_ = names.map(lambda x: (x.split(' "')[0], x.split(' "')[1]))

In [80]:
names_.take(5)

[('1', '24-HOUR MAN/EMMANUEL"'),
 ('2', '3-D MAN/CHARLES CHAN"'),
 ('3', '4-D MAN/MERCURIO"'),
 ('4', '8-BALL/"'),
 ('5', 'A"')]

In [140]:
names_df = names_.toDF()

In [141]:
names_df.show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  1|24-HOUR MAN/EMMAN...|
|  2|3-D MAN/CHARLES C...|
|  3|   4-D MAN/MERCURIO"|
|  4|            8-BALL/"|
|  5|                  A"|
|  6|              A'YIN"|
|  7|       ABBOTT, JACK"|
|  8|            ABCISSA"|
|  9|               ABEL"|
| 10|ABOMINATION/EMIL ...|
| 11|ABOMINATION | MUT...|
| 12|        ABOMINATRIX"|
| 13|            ABRAXAS"|
| 14|         ADAM 3,031"|
| 15|            ABSALOM"|
| 16|ABSORBING MAN/CAR...|
| 17|ABSORBING MAN | M...|
| 18|               ACBA"|
| 19|ACHEBE, REVEREND ...|
| 20|           ACHILLES"|
+---+--------------------+
only showing top 20 rows



In [143]:
most_popular_name = names_df.filter(F.col("_1") == most_popular_id).select("_2").first()[0]

In [144]:
most_popular_name

'CAPTAIN AMERICA"'

<font color='green'>**Fin ejercicio 1**</font>

In [149]:
wo_mp = ordered_id.filter(ordered_id['_2'] != most_popular_id)
wo_mp.show()

+----+----+
|  _2|suma|
+----+----+
|5306|1745|
|2664|1532|
|5716|1429|
|6306|1397|
|3805|1389|
|2557|1374|
|4898|1348|
|5736|1292|
| 403|1283|
|6066|1266|
|2650|1247|
|2399|1179|
|1289|1107|
|5467|1098|
| 133|1097|
|6148|1096|
| 154|1095|
|5046|1083|
|1602|1082|
|2548|1069|
+----+----+
only showing top 20 rows



In [152]:
second = wo_mp.first()[0]

In [153]:
second

'5306'

In [157]:
second_name = names_df.filter(F.col("_1") == second).select("_2").first()[0].split("/")[0]

In [158]:
second_name

'SPIDER-MAN'